# Reading Stata Dictionaries

In [ ]:
import re
from typing import List, Tuple
from dataclasses import dataclass, field

In [ ]:
import pandas as pd

Take a look at the data

We need to read in the information about the schema from a collection of lines that typically look like this:
    
    _column(1)     str12   caseid  %12s  "RESPONDENT ID NUMBER"
    _column(13)    byte    rscrinf  %1f  "WHETHER R IS SCREENER INFORMANT"
    

Each line contains 5 pieces of information:

- Where the column starts in the fixed width file (columns 1 and 13 in this case)
- The type of data (strings of 12 characters and bytes)
- The name of the column
- A more compact description of the data type as a format string
- A description of the column

The need to extract this information from lines of this type, whilst ignoring others

First off, we need to identify the lines we're interested in - i.e those with `_column([start])` in them and at the same time capture the actual start value in the bracketed portion

In [ ]:
COLUMN_IDENTIFIER = re.compile(r'_column\(([\d]+)\)')

In [ ]:
line = '    _column(13)    byte    rscrinf   %1f  "WHETHER R IS SCREENER INFORMANT"'

We are only interested in part of the string so we need to use `search` rather than `match`

In [ ]:
m = COLUMN_IDENTIFIER.search(line)

In [ ]:
m.groups()

And we want the numerical value in the single group

In [ ]:
width = int(m.groups()[0])

And use this information to work out where the rest of the string starts after the match

In [ ]:
# start and end positions of the first matching group
m.span(0)
_, start = m.span(0)

Next we need to extract the quoted long description

In [ ]:
QUOTED_VALUE = re.compile(r'"([^"]+)"')

In [ ]:
m = QUOTED_VALUE.search(line)

In [ ]:
description = m.groups()[0]

An find out the position in the string *before* the match

In [ ]:
end, _ = m.span(0)

Next we need to extract the rest of the information from the part of the string between the two matches

In [ ]:
vtype, name, fstring = line[start:end].split()
print(f'Type: {vtype}, Name: {name}, Format: {fstring}')

Next we need to map stata vtypes to python types

In [ ]:
TYPE_MAP = dict(
    byte=int,
    int=int,
    long=int,
    float=float, 
    double=float,
    numeric=float
)

In [ ]:
# map vtype to python type
vtype = TYPE_MAP.get(vtype, str)

Now we need a data class to contain these related field

In [ ]:
@dataclass
class Column:
    # start position
    start: int
    # end position - we don't know this at first
    end: int = field(default=0, init=False)
    # type of data
    vtype: type
    # name of the column
    name: str
    fstring: str
    description: str = field(repr=False)
        
    def col_spec(self, start_index=0) -> Tuple[int, int]:
        '''
        Returns the start and end positions, with possible correction for zero or one based start
        '''
        return (self.start - start_index, self.end - start_index)

And a method that extracts a column instance from a line

In [ ]:
def read_column(line: str) -> Column:
    column_match = COLUMN_IDENTIFIER.search(line)
    description_match = QUOTED_VALUE.search(line)
    # the end of the match is the start of the line we want
    _, s_start = column_match.span(0)
    # the start of the description match is the end of the portion of the rest of the line
    s_end, _ = description_match.span(0)
    # get the captured values
    position = int(column_match.groups()[0])
    description = description_match.groups()[0]
    # get the three remaining values
    vtype, name, fstring = line[s_start:s_end].split()
    # return them
    return Column(position, TYPE_MAP.get(vtype, str), name, fstring, description)
    

In [ ]:
col = read_column(line)
print(col)

Now we can read in a stata schema, given a filepath

In [ ]:
def read_stata_dictionary(filepath) -> List[Column]:
    columns = []
    # record both starting position of each column
    with open(filepath) as fp:
        for line in fp:
            if '_column' not in line:
                # doesn't contain any data
                continue
            columns.append(read_column(line))
    # work out the end positions. Start with all columns except the first
    for i in range(1, len(columns)):
        # [start, end), e.g [1, 13],[13, 14]..
        columns[i-1].end = columns[i].start
        
    return columns

In [ ]:
columns = read_stata_dictionary('../data/2002FemResp.dct')

In [ ]:
print(f'{len(columns)} Columns')

In [ ]:
for column in columns[:5]:
    print(column)
print('...')
for column in columns[-5:]:
    print(column)

We can now try reading in a fixed with file using [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_fwf.html)

In [ ]:
# counting starts from 0, so we will need to extract 1 from the (start, end) positions
index_base=1
col_specs = [c.col_spec(index_base) for c in columns]
col_names = [c.name for c in columns]

In [ ]:
df = pd.read_fwf(
    '../data/2002FemResp.dat.gz',
    colspecs=col_specs,
    names=col_names,
    compression='gzip',
    nrows=100
)

In [ ]:
# column names
df.columns

In [ ]:
df.dtypes

In [ ]:
df.head()

Lets encapulate this in a data reading method

In [ ]:
def read_fixed_width(dct_file: str, data_file: str, nrows=None) -> pd.DataFrame:
    # options to pass to read_fwf
    options = {'nrows': nrows}
    # is it compressed
    if data_file.endswith('.gz'):
        options['compression'] = 'gzip'
    columns = read_stata_dictionary(dct_file)
    # zero based indexing
    index_base=1
    col_specs = [c.col_spec(index_base) for c in columns]
    col_names = [c.name for c in columns]
    return pd.read_fwf(
        data_file,
        colspecs=col_specs,
        names=col_names,
        **options
    )

In [ ]:
df = read_fixed_width(
    '../data/2002FemResp.dct',
    '../data/2002FemResp.dat.gz',
    nrows=100
)

In [ ]:
df.head()

Lets put this in its own dedicated module

In [ ]:
import sys
sys.path.append('../lib')

In [ ]:
import fwf

In [ ]:
df = fwf.read_fixed_width(
    '../data/2002FemResp.dct',
    '../data/2002FemResp.dat.gz'
)

In [ ]:
df.head()

In [ ]:
df.to_feather('../data/2002FemResp.feather')